In [1]:
from googleapiclient.discovery import build
import json
import pandas as pd
from pytube import extract
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound
# from googletrans import Translator

In [2]:
with open("youtube_api_key.txt","r") as file:
    api_key = file.read().strip()

In [3]:
channel_id = "UC-Lq6oBPTgTXT_K-ylWL6hg"

In [4]:
# Build the YouTube API client
youtube = build("youtube", "v3", developerKey=api_key)

In [5]:
def get_channel_videos(channel_id):
    # Retrieve the uploads playlist ID
    request = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    )
    response = request.execute()
    uploads_playlist_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    # Retrieve all videos in the uploads playlist
    videos = []
    next_page_token = None
    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        videos += response["items"]
        next_page_token = response.get("nextPageToken")
        if next_page_token is None:
            break

    return videos

In [6]:
def get_video_descriptions(videos):
    descriptions = []
    for video in videos:
        video_id = video["snippet"]["resourceId"]["videoId"]
        title = video["snippet"]["title"]
        description = video["snippet"]["description"]
        descriptions.append({"title": title, "description": description, "video_id": video_id})
    return descriptions

In [7]:
# Get all videos from the channel
videos = get_channel_videos(channel_id)

# Get descriptions of all videos
descriptions = get_video_descriptions(videos)

In [8]:
videos[0]

{'kind': 'youtube#playlistItem',
 'etag': 'yl6tDlzRVjwsFXHgzyfWzrQp0sQ',
 'id': 'VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLklRb1ZPcXdQWlpB',
 'snippet': {'publishedAt': '2024-06-12T06:30:10Z',
  'channelId': 'UC-Lq6oBPTgTXT_K-ylWL6hg',
  'title': 'This Couple Brings Pune Street Food To Bengaluru! PUNERI VADE Vada Pav, Misal, Poha, Kanda Bhaji...',
  'description': '#punerivade #vadapav #bangalore \n\npune vada pav in bengaluru, vada pav, bhajji pav, street food, bangalore, puneri vade, kudlu road, muneshwara nagar, hsr layout, support small business, bengaluru food tour, bangalore food walk, bangalore food trail, karnataka food tour, support local, support small business, gourmet on the road, kripal amanna, food lovers tv, food lovers india, truth in food \n\nEvery once in a while, my heart yearns for a good vada pav. And so when I hear of one, I lose no time in seeking it out. And so it was in this context that I found myself at Puneri Vade, a tiny nook in the wall near HSR layout where I met 

In [9]:
descriptions[0]

{'title': 'This Couple Brings Pune Street Food To Bengaluru! PUNERI VADE Vada Pav, Misal, Poha, Kanda Bhaji...',
 'description': '#punerivade #vadapav #bangalore \n\npune vada pav in bengaluru, vada pav, bhajji pav, street food, bangalore, puneri vade, kudlu road, muneshwara nagar, hsr layout, support small business, bengaluru food tour, bangalore food walk, bangalore food trail, karnataka food tour, support local, support small business, gourmet on the road, kripal amanna, food lovers tv, food lovers india, truth in food \n\nEvery once in a while, my heart yearns for a good vada pav. And so when I hear of one, I lose no time in seeking it out. And so it was in this context that I found myself at Puneri Vade, a tiny nook in the wall near HSR layout where I met a shy but friendly young couple, Akshay and Ruchika Shinde, who moved from Pune to Bengaluru a couple of years ago to introduce the vada pav, bhajjis, poha and misal of their city to Bengaluru. I tasted some of their fare, and su

In [10]:
def get_video_details(videos):
    video_details = []
    for video in videos:
        video_id = video["snippet"]["resourceId"]["videoId"]
        title = video["snippet"]["title"]
        description = video["snippet"]["description"]
        link = f"https://www.youtube.com/watch?v={video_id}"
        video_details.append({"Title": title, "Link": link, "Description": description})
    return video_details

In [11]:
video_details = get_video_details(videos)

In [12]:
df = pd.DataFrame(video_details)
df.head()

,Title,Link,Description
0,This Couple Brings Pune Street Food To Bengalu...,https://www.youtube.com/watch?v=IQoVOqwPZZA,#punerivade #vadapav #bangalore \n\npune vada ...
1,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...
2,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...
3,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...
4,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...


In [13]:
df["Description"][:10]

0    #punerivade #vadapav #bangalore \n\npune vada ...
1    Watch the complete episode at https://youtu.be...
2    From millet offerings in pongals to bisi bele ...
3    #dhanyamcafe #millets #bangalore\n \ndhanyam c...
4    #hotelgoldenriviera #pizza #mysore\n \nherbs o...
5    Namaskara folks! Hoping to shine a bit of ligh...
6    The chefs at MTR Foods created a record breaki...
7    Watch the complete video of this inspiring sto...
8    Only about three years old, however Namma Hall...
9    #tajhotelbiryani #shivajimilitaryhotelbiryani ...
Name: Description, dtype: object

In [14]:
print(df["Description"][468])

#wheelchairaccessible #gourmetontheroad #kripalamanna  

wheelchair accessible, hashtag, facebook, instagram, twitter, youtube, wheelchair accessible restaurants, eateries, food lovers tv 

Great food should be physically accessible to all food lovers, shouldn’t it? But the sad truth is, it isn’t! I recently received a message from one of our viewers speaking from experience of the challenges that those in wheelchairs face, trying to manoeuvre from their vehicle on the street onto the sidewalk, the premises, and then to the eatery. To help the community of wheelchair users, can you try and do one simple thing. 

When you put up a restaurant post on Instagram, Facebook, Twitter or any other social media, include the hashtag #wheelchairaccessible in your post in case the restaurant is.

Let’s try and help build a resource of #wheelchairaccessible eateries for those who rely on wheelchairs for mobility. Do share, repost and help spread the word. 🙏🙏

#gourmetontheroad #kripalamanna #foodlo

In [15]:
pd.DataFrame(videos[:5])

,kind,etag,id,snippet
0,youtube#playlistItem,yl6tDlzRVjwsFXHgzyfWzrQp0sQ,VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLklRb1ZPcXdQWlpB,"{'publishedAt': '2024-06-12T06:30:10Z', 'chann..."
1,youtube#playlistItem,6hKdUkdYvl6AytY04kBaZWk05Jw,VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLkJmUDAtSjA4T29Z,"{'publishedAt': '2024-06-11T06:30:04Z', 'chann..."
2,youtube#playlistItem,hsI2Bf1sIznJfNzN2iG6hj3XJiA,VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLnNvQU9xSS1kQ21N,"{'publishedAt': '2024-06-10T06:30:25Z', 'chann..."
3,youtube#playlistItem,cIVAEk6XzywjdfTN3fHHzQfK4Ww,VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLlEtak5mRGFZdU1j,"{'publishedAt': '2024-06-08T05:30:13Z', 'chann..."
4,youtube#playlistItem,FK8wpmkDRUAQMfJgS3b-E8dY2n0,VVUtTHE2b0JQVGdUWFRfSy15bFdMNmhnLmVNZ1J3ZTF2ZVBn,"{'publishedAt': '2024-06-07T06:30:11Z', 'chann..."


In [16]:
import re
def extract_links(text):
    # Regular expression to find URLs
    url_pattern = re.compile(r'(https?://\S+)')
    return url_pattern.findall(text)

In [17]:
df["Links"] = df["Description"].apply(extract_links)

In [18]:
df.head()

,Title,Link,Description,Links
0,This Couple Brings Pune Street Food To Bengalu...,https://www.youtube.com/watch?v=IQoVOqwPZZA,#punerivade #vadapav #bangalore \n\npune vada ...,"[https://www.youtube.com/c/KripalAmannaVlogs, ..."
1,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,[https://youtu.be/m2G-fnkoaOY]
2,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"[https://youtu.be/Q-jNfDaYuMc, https://maps.ap..."
3,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,"[https://www.youtube.com/c/KripalAmannaVlogs, ..."
4,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,"[https://www.youtube.com/c/KripalAmannaVlogs, ..."


In [19]:
def filter_gmaps_links(links):
    gmaps_pattern = re.compile(r'(https?://(?:goo\.gl/maps|maps\.google|google\.com/maps|maps\.app\.goo\.gl|g\.page)\S+)')
    return [link for link in links if gmaps_pattern.match(link)]

In [20]:
df['gmaps_links'] = df['Links'].apply(filter_gmaps_links)
df.head()

,Title,Link,Description,Links,gmaps_links
0,This Couple Brings Pune Street Food To Bengalu...,https://www.youtube.com/watch?v=IQoVOqwPZZA,#punerivade #vadapav #bangalore \n\npune vada ...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2GtbNmjb64a73joK9]
1,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,[https://youtu.be/m2G-fnkoaOY],[]
2,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"[https://youtu.be/Q-jNfDaYuMc, https://maps.ap...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8]
3,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8]
4,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96]


In [21]:
df['gmaps_links'] = df['gmaps_links'].apply(lambda x: x if x else None)

In [23]:
df.head()

,Title,Link,Description,Links,gmaps_links
0,This Couple Brings Pune Street Food To Bengalu...,https://www.youtube.com/watch?v=IQoVOqwPZZA,#punerivade #vadapav #bangalore \n\npune vada ...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2GtbNmjb64a73joK9]
1,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,[https://youtu.be/m2G-fnkoaOY],None
2,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"[https://youtu.be/Q-jNfDaYuMc, https://maps.ap...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8]
3,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8]
4,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96]


In [24]:
# Function to extract video ID
def get_video_id(link):
    return extract.video_id(link)

# Apply the function to the Link column
df['video_id'] = df['Link'].apply(get_video_id)

In [25]:
df[df["gmaps_links"].isnull()][["Description"]].to_csv("no_gmaps_links.csv", index=False)

In [26]:
def extract_and_translate_transcript(youtube_video_url):
    try:
        video_id = youtube_video_url.split("=")[1]
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        
        # Attempt to get a manually created transcript in English
        try:
            transcript = transcript_list.find_transcript(['en'])
        except NoTranscriptFound:
            # Attempt to get an auto-generated transcript in English
            try:
                transcript = transcript_list.find_generated_transcript(['en'])
            except NoTranscriptFound:
                # Attempt to get a manually created transcript in Hindi or Kannada
                try:
                    transcript = transcript_list.find_transcript(['hi', 'kn'])
                except NoTranscriptFound:
                    return "No transcript available"
        
        transcript_text = transcript.fetch()
        
        full_transcript = ""
        for i in transcript_text:
            full_transcript += " " + i["text"]
        
        # If the transcript is not in English, translate it
        if transcript.language_code not in ['en']:
            translator = Translator()
            translated_transcript = translator.translate(full_transcript, src=transcript.language_code, dest='en')
            return translated_transcript.text.strip()
        
        return full_transcript.strip()
    except Exception as e:
        return f"Error: {e}"

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        1048 non-null   object
 1   Link         1048 non-null   object
 2   Description  1048 non-null   object
 3   Links        1048 non-null   object
 4   gmaps_links  600 non-null    object
 5   video_id     1048 non-null   object
dtypes: object(6)
memory usage: 49.3+ KB


In [28]:
def process_row(row):
    print(f"Processing row {row.name}")
    return extract_and_translate_transcript(row['Link'])

df['transcript'] = df.apply(process_row, axis=1)

Processing row 0
Processing row 1
Processing row 2
Processing row 3
Processing row 4
Processing row 5
Processing row 6
Processing row 7
Processing row 8
Processing row 9
Processing row 10
Processing row 11
Processing row 12
Processing row 13
Processing row 14
Processing row 15
Processing row 16
Processing row 17
Processing row 18
Processing row 19
Processing row 20
Processing row 21
Processing row 22
Processing row 23
Processing row 24
Processing row 25
Processing row 26
Processing row 27
Processing row 28
Processing row 29
Processing row 30
Processing row 31
Processing row 32
Processing row 33
Processing row 34
Processing row 35
Processing row 36
Processing row 37
Processing row 38
Processing row 39
Processing row 40
Processing row 41
Processing row 42
Processing row 43
Processing row 44
Processing row 45
Processing row 46
Processing row 47
Processing row 48
Processing row 49
Processing row 50
Processing row 51
Processing row 52
Processing row 53
Processing row 54
Processing row 55
Pr

In [29]:
df.head()

,Title,Link,Description,Links,gmaps_links,video_id,transcript
0,This Couple Brings Pune Street Food To Bengalu...,https://www.youtube.com/watch?v=IQoVOqwPZZA,#punerivade #vadapav #bangalore \n\npune vada ...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2GtbNmjb64a73joK9],IQoVOqwPZZA,Error: name 'Translator' is not defined
1,A Biryani Is An Emotion! #shorts #biryani #foo...,https://www.youtube.com/watch?v=BfP0-J08OoY,Watch the complete episode at https://youtu.be...,[https://youtu.be/m2G-fnkoaOY],None,BfP0-J08OoY,to us Biryani is not just meat and rice and sp...
2,Delicious Must-Try Millet Dishes At This Small...,https://www.youtube.com/watch?v=soAOqI-dCmM,From millet offerings in pongals to bisi bele ...,"[https://youtu.be/Q-jNfDaYuMc, https://maps.ap...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8],soAOqI-dCmM,Error: name 'Translator' is not defined
3,She Wants To Serve Healthy Traditional Tiffin ...,https://www.youtube.com/watch?v=Q-jNfDaYuMc,#dhanyamcafe #millets #bangalore\n \ndhanyam c...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/2X6idFktNJv8D4CW8],Q-jNfDaYuMc,Error: name 'Translator' is not defined
4,"This Family Brings ""Healthier No Maida Pizza"" ...",https://www.youtube.com/watch?v=eMgRwe1vePg,#hotelgoldenriviera #pizza #mysore\n \nherbs o...,"[https://www.youtube.com/c/KripalAmannaVlogs, ...",[https://maps.app.goo.gl/wjg9HzcEmmcnD2Y96],eMgRwe1vePg,Error: name 'Translator' is not defined


In [30]:
df.to_csv("youtube_videos.csv", index=False)